In [15]:
import cv2
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

In [19]:
with Image.open(r'D:\SHUBHAM\CCBD\S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623_RGB.tif') as img:
    print(img.tag)
    #meta_dict = {TAGS[key] : img.tag[key] for key in img.tag.iterkeys()}

{256: (10980,), 257: (10980,), 258: (8, 8, 8, 8), 259: (1,), 262: (2,), 273: (88293, 132213, 176133, 220053, 263973, 307893, 351813, 395733, 439653, 483573, 527493, 571413, 615333, 659253, 703173, 747093, 791013, 834933, 878853, 922773, 966693, 1010613, 1054533, 1098453, 1142373, 1186293, 1230213, 1274133, 1318053, 1361973, 1405893, 1449813, 1493733, 1537653, 1581573, 1625493, 1669413, 1713333, 1757253, 1801173, 1845093, 1889013, 1932933, 1976853, 2020773, 2064693, 2108613, 2152533, 2196453, 2240373, 2284293, 2328213, 2372133, 2416053, 2459973, 2503893, 2547813, 2591733, 2635653, 2679573, 2723493, 2767413, 2811333, 2855253, 2899173, 2943093, 2987013, 3030933, 3074853, 3118773, 3162693, 3206613, 3250533, 3294453, 3338373, 3382293, 3426213, 3470133, 3514053, 3557973, 3601893, 3645813, 3689733, 3733653, 3777573, 3821493, 3865413, 3909333, 3953253, 3997173, 4041093, 4085013, 4128933, 4172853, 4216773, 4260693, 4304613, 4348533, 4392453, 4436373, 4480293, 4524213, 4568133, 4612053, 4655973,

C:\ProgramData\Anaconda31\lib\site-packages\PIL\Image.py:2371: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


In [16]:
class ImageMetaData(object):
    '''
    Extract the exif data from any image. Data includes GPS coordinates, 
    Focal Length, Manufacture, and more.
    '''
    exif_data = None
    image = None

    def __init__(self, img_path):
        self.image = Image.open(img_path)
        #print(self.image._getexif())
        self.get_exif_data()
        super(ImageMetaData, self).__init__()

    def get_exif_data(self):
        """Returns a dictionary from the exif data of an PIL Image item. Also converts the GPS Tags"""
        exif_data = {}
        info = self.image._getexif()
        if info:
            for tag, value in info.items():
                decoded = TAGS.get(tag, tag)
                if decoded == "GPSInfo":
                    gps_data = {}
                    for t in value:
                        sub_decoded = GPSTAGS.get(t, t)
                        gps_data[sub_decoded] = value[t]

                    exif_data[decoded] = gps_data
                else:
                    exif_data[decoded] = value
        self.exif_data = exif_data
        return exif_data

    def get_if_exist(self, data, key):
        if key in data:
            return data[key]
        return None

    def convert_to_degress(self, value):

        """Helper function to convert the GPS coordinates 
        stored in the EXIF to degress in float format"""
        d0 = value[0][0]
        d1 = value[0][1]
        d = float(d0) / float(d1)

        m0 = value[1][0]
        m1 = value[1][1]
        m = float(m0) / float(m1)

        s0 = value[2][0]
        s1 = value[2][1]
        s = float(s0) / float(s1)

        return d + (m / 60.0) + (s / 3600.0)

    def get_lat_lng(self):
        """Returns the latitude and longitude, if available, from the provided exif_data (obtained through get_exif_data above)"""
        lat = None
        lng = None
        exif_data = self.get_exif_data()
        #print(exif_data)
        if "GPSInfo" in exif_data:      
            gps_info = exif_data["GPSInfo"]
            gps_latitude = self.get_if_exist(gps_info, "GPSLatitude")
            gps_latitude_ref = self.get_if_exist(gps_info, 'GPSLatitudeRef')
            gps_longitude = self.get_if_exist(gps_info, 'GPSLongitude')
            gps_longitude_ref = self.get_if_exist(gps_info, 'GPSLongitudeRef')
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat = self.convert_to_degress(gps_latitude)
                if gps_latitude_ref != "N":                     
                    lat = 0 - lat
                lng = self.convert_to_degress(gps_longitude)
                if gps_longitude_ref != "E":
                    lng = 0 - lng
        return lat, lng

In [17]:
path_name = r'D:\SHUBHAM\CCBD\S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623_RGB.tif'
meta_data =  ImageMetaData(path_name)
latlng =meta_data.get_lat_lng()
print(latlng)
exif_data = meta_data.get_exif_data()
print(exif_data)


C:\ProgramData\Anaconda31\lib\site-packages\PIL\Image.py:2371: DecompressionBombWarning: Image size (120560400 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


AttributeError: 'TiffImageFile' object has no attribute '_getexif'

In [14]:
file_path = r'D:\SHUBHAM\CCBD\S2A_MSIL2A_20190104T051211_N0211_R019_T43PGQ_20190104T094623_RGB.tif'    
gps = getGPS(file_path)
print(gps)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe4 in position 19: invalid continuation byte

In [6]:
# Print out just GPS Data of interest
for tag in data.keys():
    print("%s: %s",tag, data[tag])

# Print out raw GPS Data for debugging
for tag in rawData.keys():
    print ("%s: %s", tag, rawData[tag])



In [ ]:
# Create a GPSPhoto Object
photo = gpsphoto.GPSPhoto()
photo = gpsphoto.GPSPhoto("/path/to/photo.jpg")

# Create GPSInfo Data Object
info = gpsphoto.GPSInfo((35.104860, -106.628915))
info = gpsphoto.GPSInfo((35.104860, -106.628915), \
          timeStamp='1970:01:01 09:05:05')
info = gpsphoto.GPSInfo((35.104860, -106.628915), \
          alt=10, timeStamp='1970:01:01 09:05:05')

# Modify GPS Data
photo.modGPSData(info, '/path/to/newFile.jpg')

# Strip GPS Data
photo.stripData('/path/to/newFile.jpg')